In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muhammadshahidazeem/customer-churn-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/customer-churn-dataset


In [3]:
import os
data_path= "/kaggle/input/customer-churn-dataset"
print(os.listdir(data_path))

['customer_churn_dataset-testing-master.csv', 'customer_churn_dataset-training-master.csv']


In [4]:
# Import the all necessary Library and tools 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder



2025-09-16 19:43:09.474869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758051789.668135      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758051789.723273      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
data=pd.read_csv("/kaggle/input/customer-churn-dataset/customer_churn_dataset-training-master.csv")

In [6]:
data

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
440828,449995.0,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
440829,449996.0,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
440830,449997.0,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
440831,449998.0,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [7]:
data['Churn'].value_counts()

Churn
1.0    249999
0.0    190833
Name: count, dtype: int64

# Let's Preprocess the Datast 

In [8]:
# check null data
data.isnull().sum()


CustomerID           1
Age                  1
Gender               1
Tenure               1
Usage Frequency      1
Support Calls        1
Payment Delay        1
Subscription Type    1
Contract Length      1
Total Spend          1
Last Interaction     1
Churn                1
dtype: int64

In [9]:
data.dropna(inplace=True)

In [10]:
data.isnull().sum()

CustomerID           0
Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64

In [11]:
data.duplicated().sum()

0

In [12]:
# Drop the Customer ID, no needed
data.drop(columns=['CustomerID'],inplace=True)

In [13]:
data

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
440828,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
440829,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
440830,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
440831,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [14]:
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# let's check about the categorical Column

In [15]:
data['Subscription Type'].value_counts()

Subscription Type
Standard    149128
Premium     148678
Basic       143026
Name: count, dtype: int64

In [16]:
data['Contract Length'].value_counts()

Contract Length
Annual       177198
Quarterly    176530
Monthly       87104
Name: count, dtype: int64

In [17]:
data['Gender'].value_counts()

Gender
Male      250252
Female    190580
Name: count, dtype: int64

# Let's Encode the Categorical Columns

In [18]:
cat_cols= ["Subscription Type", "Contract Length", "Gender"]

In [19]:
trans = ColumnTransformer(
[
("cat", OneHotEncoder(drop="first"), cat_cols)
    
], remainder="passthrough"


    
)

In [20]:
data_trans = trans.fit_transform(data)

In [21]:
data_trans

array([[  1.,   0.,   0., ..., 589.,  13.,   1.],
       [  0.,   0.,   0., ..., 158.,  12.,   1.],
       [  0.,   1.,   0., ..., 915.,  26.,   1.],
       ...,
       [  0.,   0.,   1., ..., 979.,   8.,   1.],
       [  1.,   0.,   0., ..., 824.,   1.,   1.],
       [  0.,   1.,   0., ..., 951.,  19.,   1.]])

In [22]:
cat_names= trans.named_transformers_["cat"].get_feature_names_out(cat_cols)

In [23]:
cat_names

array(['Subscription Type_Premium', 'Subscription Type_Standard',
       'Contract Length_Monthly', 'Contract Length_Quarterly',
       'Gender_Male'], dtype=object)

In [24]:
num_cols = []
for i in data.columns:
    if i not in cat_cols:
        num_cols.append(i)

In [25]:
num_cols

['Age',
 'Tenure',
 'Usage Frequency',
 'Support Calls',
 'Payment Delay',
 'Total Spend',
 'Last Interaction',
 'Churn']

In [26]:
all_cols = list(cat_names) + num_cols

In [27]:
all_cols

['Subscription Type_Premium',
 'Subscription Type_Standard',
 'Contract Length_Monthly',
 'Contract Length_Quarterly',
 'Gender_Male',
 'Age',
 'Tenure',
 'Usage Frequency',
 'Support Calls',
 'Payment Delay',
 'Total Spend',
 'Last Interaction',
 'Churn']

In [28]:
transformed_data = pd.DataFrame(data_trans, columns=all_cols)

In [29]:
transformed_data

,Subscription Type_Premium,Subscription Type_Standard,Contract Length_Monthly,Contract Length_Quarterly,Gender_Male,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn
0,1.0,0.0,0.0,1.0,0.0,23.0,10.0,16.0,8.0,16.0,589.00,13.0,1.0
1,0.0,0.0,0.0,1.0,1.0,33.0,3.0,20.0,0.0,22.0,158.00,12.0,1.0
2,0.0,1.0,0.0,0.0,0.0,27.0,57.0,3.0,0.0,24.0,915.00,26.0,1.0
3,0.0,0.0,0.0,0.0,0.0,35.0,28.0,23.0,5.0,13.0,334.00,29.0,1.0
4,0.0,0.0,0.0,0.0,1.0,28.0,53.0,30.0,2.0,5.0,568.86,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440827,1.0,0.0,0.0,0.0,1.0,41.0,44.0,18.0,2.0,13.0,717.39,3.0,0.0
440828,0.0,0.0,0.0,0.0,0.0,45.0,44.0,14.0,3.0,13.0,995.38,8.0,0.0
440829,0.0,0.0,1.0,0.0,0.0,37.0,9.0,4.0,10.0,12.0,979.00,8.0,1.0
440830,1.0,0.0,0.0,0.0,1.0,33.0,27.0,5.0,10.0,20.0,824.00,1.0,1.0


# Let's split the Dataset 

In [30]:
x = transformed_data.drop(columns=['Churn'])
y = transformed_data['Churn']


In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [32]:
x_train

,Subscription Type_Premium,Subscription Type_Standard,Contract Length_Monthly,Contract Length_Quarterly,Gender_Male,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction
321744,1.0,0.0,0.0,0.0,1.0,50.0,20.0,21.0,4.0,14.0,677.51,25.0
175119,0.0,0.0,0.0,0.0,1.0,49.0,7.0,16.0,3.0,13.0,806.60,9.0
272447,0.0,0.0,0.0,0.0,1.0,42.0,30.0,16.0,4.0,20.0,825.52,14.0
66769,0.0,0.0,0.0,0.0,1.0,33.0,57.0,1.0,3.0,5.0,732.27,17.0
70297,0.0,1.0,0.0,1.0,0.0,28.0,55.0,12.0,1.0,14.0,296.00,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
259178,0.0,1.0,0.0,1.0,1.0,47.0,26.0,3.0,1.0,0.0,565.38,9.0
365838,1.0,0.0,0.0,1.0,0.0,49.0,31.0,2.0,1.0,10.0,622.01,2.0
131932,0.0,0.0,0.0,0.0,0.0,18.0,30.0,12.0,2.0,14.0,580.00,20.0
146867,0.0,0.0,1.0,0.0,0.0,19.0,51.0,22.0,6.0,3.0,933.00,12.0


In [33]:
y_train.value_counts()

Churn
1.0    199961
0.0    152704
Name: count, dtype: int64

# Now let's Scale the Dataset 

In [34]:
scale = StandardScaler()

In [35]:
x_train = scale.fit_transform(x_train)
x_test  = scale.transform(x_test)

In [36]:
x_train

array([[ 1.3990297 , -0.71432383, -0.49641307, ...,  0.1249753 ,
         0.18896025,  1.22472461],
       [-0.71478111, -0.71432383, -0.49641307, ...,  0.00386346,
         0.72472469, -0.63604636],
       [-0.71478111, -0.71432383, -0.49641307, ...,  0.85164635,
         0.80324869, -0.05455543],
       ...,
       [-0.71478111, -0.71432383, -0.49641307, ...,  0.1249753 ,
        -0.21573718,  0.64323369],
       [-0.71478111, -0.71432383,  2.0144514 , ..., -1.20725495,
         1.24932477, -0.2871518 ],
       [-0.71478111, -0.71432383,  2.0144514 , ..., -0.23836022,
         0.85919496, -1.21753729]])

# Now let's make a Machine Learning Model 

In [37]:
# we will use logistic regression 
from sklearn.linear_model import LogisticRegression

In [38]:
model = LogisticRegression(max_iter=1000)

In [39]:
model

LogisticRegression(max_iter=1000)

In [40]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [41]:
pred_ml=model.predict(x_test)

In [42]:
acc= accuracy_score(pred_ml,y_test)
acc

0.8934862250048204

In [43]:
# now Let's use Random Forest algo
from sklearn.ensemble import RandomForestClassifier

In [44]:
model_ml2 = RandomForestClassifier()

In [45]:
model_ml2


RandomForestClassifier()

In [46]:
model_ml2.fit(x_train,y_train)


RandomForestClassifier()

In [47]:
pred2 = model_ml2.predict(x_test)
acc2= accuracy_score(pred2,y_test)

In [48]:
acc2

0.9997277893089251

# Now We will Build the ANN model ( Deep Learning Algorithm)

In [51]:
# Let's import the tensorflow library for Neural network

In [53]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [55]:
x_train.shape[1]

12

In [65]:
# Let's make a model ( 3 layers )
n=x_train.shape[1]
model = keras.Sequential(
[
layers.Dense(64,activation='relu',input_shape=(n,)),
     layers.Dense(32, activation='relu'),
    
    layers.Dense(16, activation='relu'),
      layers.Dense(4, activation='relu'),
    layers.Dense(1,activation='sigmoid')
   
]
    
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
# complie :
model.compile(
optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    
)


In [67]:
history = model.fit(x_train, y_train, epochs=100, batch_size=100, verbose=1)

Epoch 1/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9452 - loss: 0.1347
Epoch 2/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9905 - loss: 0.0316
Epoch 3/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9932 - loss: 0.0239
Epoch 4/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9946 - loss: 0.0193
Epoch 5/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9954 - loss: 0.0167
Epoch 6/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9955 - loss: 0.0150
Epoch 7/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9961 - loss: 0.0129
Epoch 8/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9962 - loss: 0.0122
Epoch 9/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9963 - loss: 0.0110
Epoch 10/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9966 - loss: 0.0104
Epoch 11/100
3527/3527 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9972 - loss: 0.0087
Epoch 12/100
3527/

In [68]:
loss,ac_nn = model.evaluate(x_test,y_test,verbose=0)
print("Test Accuracy : {:.2f}".format(ac_nn))

Test Accuracy : 1.00
